# **News Scrapper**
News Sources: Sputnik - Tass - SCMP - Xinhua

In [24]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [25]:
import time
import collections
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--incognito")
import collections

### Google Drive Mount

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Scrapper parent class: NewsExtractor

In [27]:


class NewsExtractor:
  articles=collections.deque()
  dir=""

  def setdirpath(self, path):
      self.dir=path
  def downloadarticles(self, i, array1):
      #print(" => downloading articles. .  .  .    . thread: ", i)
      #from tqdm import tqdm
      le=len(array1)
      #pbar = tqdm(total=len(array))
      print(" threade:  => ", i)
      c=0
      #d=webdriver.Chrome('chromedriver',chrome_options=chrome_options) #Path to Chrome Driver
      for q in array1:
          print(" threade:", i, "  => (",  c,"/",le,")")
          x=q
          (s, l, De, Dp, t, aut, art)=self.getarticle(x)
          if (s+l+De+Dp+t+aut+art!="0"+"0"+"0"+"0"+"0"+"0"+"0"):
              non=['/',':','*','?','"','<','>','|']
              name=s+" _ "+Dp+" _ "+t+".txt"
              for n in non:
                  name=name.replace(n, '-')
              #from pathlib import Path
              #Path(self.dir+Dp+"/").mkdir(parents=True, exist_ok=True)
              #f = open(self.dir+Dp+"/"+name, "w", encoding="utf-8")
              f = open(self.dir+name, "w", encoding="utf-8")
              f.write("Source: " + s)
              f.write("\nLink: " + l)
              f.write("\nDateOfExtraction: " + De)
              f.write("\nDateOfPublication: " + Dp)
              f.write("\nTitle: " + t)
              f.write("\nAuthor: " + aut)
              f.write("\nArticle: " + art)
              f.close()
              #pbar.update(1)
              #print(" threadw:  => ", i)
          c=c+1
      #pbar.close()
      print(" => download complete! thread: ", i)
  
  def downloaddata(self, d, m, year, url, chk ):
      
      import numpy as np

      if chk==0:
        self.getdatelinks(d, m, year)
      else:
        self.getdatelinks1(url)

      #print(len( articles))
      print(" => links extracted :" + str(len(self.articles)))
      arr = np.array(self.articles)
      splitno=1
      if len( self.articles) > 1000:
          splitno=14 #change according to number of articles must be less then no of articles 
      if len( self.articles) > 500 and len( self.articles) <= 1000:
          splitno=11 #change according to number of articles must be less then no of articles 
      if len( self.articles) > 100 and len( self.articles) <= 500:
          splitno=8 #change according to number of articles must be less then no of articles  
      if len( self.articles) >20 and len( self.articles) <= 100:
          splitno=5
      if len( self.articles) <20:
          splitno=1  
      newarr = np.array_split(arr, splitno)
      import threading
      i=0
      while(i<splitno):
          a=newarr[i]
          t1 = threading.Thread(target=self.downloadarticles, args=(i,a,))
          t1.start()
          time.sleep(8)
          i=i+1
      t1.join()  
  def daily(self):
      print(" => Daily news article parser")
      date1="2021-01-01"
      date2="2021-01-01"
      c=1
      self.extract(date1, date2, c)
      
  def weekly(self):
      print(" => weekly news article parser")

      Datetoday=str(datetime.date(datetime.now()))
      today=Datetoday.split('-',2)
      year=int(today[0])
      month=int(today[1])
      day=int(today[2])
      day=day-1
      if day==0:
          if(month==3):
              day=28
          if month in [4,5,7,8,10,12]:
              day=30
              
          if month in [1,2,4,6,9,11]: 
              day=31
          month=month-1
          if(month==0):
              month=12

      print(" ----", Datetoday, "-----\n")
      #input1 from: 
      m2=month #month
      d2=day #day
      m1=month #month
      d1=day #day

      c=2
      if c==2: 
        d1=d2-7
        if d1<=0:
          if(month==3):
            day=28
          if month in [3,5,7,8,10,12]:
            day=30    
          if month in [1,2,4,6,9,11]: 
            day=31 
          d1=day+d1
          m1=month-1
          if(m1==0):
            month=12
        else:
          m1=m2  

      date1=str(year)+"-"+str(m1)+"-"+str(d1)
      date2=str(year)+"-"+str(m2)+"-"+str(d2)

      self.rangedate(date1, date2)

  def monthly(self):
      print(" => monthly news article parser")

      Datetoday=str(datetime.date(datetime.now()))
      today=Datetoday.split('-',2)
      year=int(today[0])
      month=int(today[1])
      day=int(today[2])
      day=day-1
      if day==0:
          if(month==3):
              day=28
          if month in [4,5,7,8,10,12]:
              day=30
              
          if month in [1,2,4,6,9,11]: 
              day=31
          month=month-1
          if(month==0):
              month=12
      #input1 from: 
      m2=month #month
      d2=day #day
      m1=month #month
      d1=day #day

      c=3
      if c==3:  #monthly
        m1=m2-1
        if(m1==0):
          m1=12
      
      date1=str(year)+"-"+str(m1)+"-"+str(d1)
      date2=str(year)+"-"+str(m2)+"-"+str(d2)
      self.rangedate(date1, date2)

  def getarticle(self, x):
    pass  
  def getdatelinks(self, d, m, year):
    pass 
  def extract(self, d, m, year):
    pass    
  def rangedate(self, date1, date2):
      print(" => specified news article parser")
      c=0
      self.extract(date1, date2, c)   

### Scrapper: Sputnik News

In [28]:
class Sputnik(NewsExtractor):


  def getdatelinks(self, d, m, y):
      months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] 
      if d<10:
        day="0"+str(d)
      else:
        day=str(d)
      year=str(y)  
      month=months[m-1]
      print(" => getting article links...........", day,month+",",year)
      import requests
      from bs4 import BeautifulSoup
      catagories=["search"]

      #check="/article/"
      #check1="/opinion/"
      checks=["/us/", "/uk/", "/asia/", "/africa/", "/europe/", "/latam/", "/russia/", "/india/", "/middleeast/", "/business/", "/science/", "/analysis/" ]
      split="https://sputniknews.com/"
      #/20210723
      datechk="/"+year+month+str(day)
      for c in catagories:
        url="https://web.archive.org/web/"+year+month+str(day)+"032150"+"/https://sputniknews.com/"
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for link in soup.find_all("a"):
          indicator=0
          for c in checks:
            if c in str(link.get('href')):
              indicator=1
          if indicator==1:
            word=str(link.get('href'))
            #print(word)
            
            wordlist=word.split(split, 1)
            if len(word)==len(wordlist):
              w=wordlist[0]
            else:  
              if len(wordlist)>1:
                #print(wordlist)
                url=split+wordlist[1]
                #print(url)
              else:
                url=wordlist
            if datechk in url and url not in self.articles:      
              print(url)
              self.articles.append(url)
            #print(url)

  def getdatelinks1(self, url):
    wd = webdriver.Chrome('chromedriver',options=options)
    #wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    #wd.maximize_window()
    wd.get("https://sputniknews.com/search/")
    
    #---------------Text articles check-------------
    wd.find_element_by_xpath(
        "/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/form/div[1]/ul/li[2]/label"
    ).click()
    #---------------dropdown timespecification-------------
    wd.find_element_by_xpath(
        "/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/form/div[1]/div/div[1]"
    ).click()
    #---------------select timespecification today-------------
    
    #li[1] -> day
    #li[2] -> week
    #li[3] -> month
    #li[4] -> year
    wd.find_element_by_xpath(url).click()
    
    
    time.sleep(4)
    print(" => getting article links...........")
    f=1 #flage
    while(f):
        links1 = wd.find_elements_by_class_name('b-plainlist__announce')
        l1=len(links1)
        el = wd.find_element_by_xpath("/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/div/div[3]/div")
        #wd.execute_script("arguments[0].scrollIntoView();", el)
        actions = ActionChains(wd)
        actions.move_to_element(el).perform()
        actions.perform()
        actions.move_by_offset(1, 1)
        actions.click()
        actions.perform()
        time.sleep(3)
        el = wd.find_element_by_xpath("/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/div/div[3]/div")
        #wd.execute_script("arguments[0].scrollIntoView();", el)
        actions = ActionChains(wd)
        actions.move_to_element(el).perform()
        actions.perform()
        actions.move_by_offset(1, 1)
        actions.click()
        actions.perform()
        time.sleep(3)
        links2 = wd.find_elements_by_class_name('b-plainlist__announce')
        l2=len(links2)
        if((l2-l1)==0):
            f=0
    
    all_links = wd.find_elements_by_class_name('b-plainlist__title')
    for link in all_links:
        links = link.find_elements_by_tag_name('a')
        for l in links:
            self.articles.append(l.get_attribute('href'))
            #print(l.get_attribute('href'))
                    
    wd.close()
    
            
  def getarticle(self, x):
    import time
    print(" => downloading..... "+x)
    d=webdriver.Chrome('chromedriver',options=options) #Path to Chrome Driver
    #d.minimize_window()
    err=0
    try:
      d.get(x)
    except selenium.common.exceptions.WebDriverException:
      err=1
    if err!=1:
      d.get(x)
      time.sleep(2)
      source="sputniknews"
      author="sputniknews"
      link=str(x)
      title = d.find_element_by_class_name('b-article__header-title ').text
  
      text=""
      all_links = d.find_elements_by_class_name('b-article__text')
      for i in all_links:
          text=text+i.text
  
      y = d.find_element_by_class_name('b-article__refs-date').text
      a=y.split(' GMT ', 1)
      b=a[1]
      c=b.split('.', 1)
      day=c[0]
      dd=c[1]
      e=dd.split('.', 1)
      month=e[0]
      year=e[1]
      timing=a[0]       
      DateOfPublication=year+"-"+month+"-"+day
  
      DateOfExtraction=str(datetime.date(datetime.now()))
      article=text
      d.close()
      return source, link, DateOfExtraction, DateOfPublication, title, author, article
    else:
      return "0","0","0","0","0","0" ,"0"      

  def extract(self, date1, date2, c):
    from datetime import datetime
    Datetoday=str(datetime.date(datetime.now()))
    print("\n => scraper: sputnik news", " - date:", Datetoday)
    

    if c==0: #defined range
      print(" to:",date1, "----- from:",date2,"\n")
      datei=date1.split('-',2)
      year=int(datei[0])
      m1=int(datei[1])
      d1=int(datei[2])
      datef=date2.split('-',2)
      year=int(datef[0])
      m2=int(datef[1])
      d2=int(datef[2])

      url=""
      chk=0
      for j in range(m1, m2+1):
          df=31
          di=1
          if j==m1:
              di=d1
          if j==m2:
              df=d2
          for i in range(di, df+1):
              self.articles.clear()
              self.downloaddata(i, j, year, url, chk)
    else:
      if c==1:  #daily
        url="/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/form/div[1]/div/ul/li[1]"
      if c==2: 
        url="/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/form/div[1]/div/ul/li[2]"
      if c==3:  #monthly
        url="/html/body/div[2]/div[8]/div[1]/div[1]/div[2]/form/div[1]/div/ul/li[3]"
      
      i=0
      j=0
      year=0
      chk=1
      self.articles.clear()  
      self.downloaddata(i, j, year, url, chk)



### Scrapper: SCMP News

In [29]:
class Scmp(NewsExtractor):
  def getdatelinks(self, d, m, y):
      months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] 
      if d<10:
        day="0"+str(d)
      else:
        day=str(d)
      year=str(y)  
      month=months[m-1]
      print(" => getting article links...........", day,month+",",year)
      import requests
      from bs4 import BeautifulSoup
      catagories=["asia", "world", "sport" ]

      check="/article/"
      check1="/opinion/"
      split="https://www.scmp.com/"
      for c in catagories:
        url="https://web.archive.org/web/"+year+month+str(day)+"032150"+"/https://www.scmp.com/news/"+c
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        for link in soup.find_all('a'):
          if check in str(link.get('href')) and check1 not in str(link.get('href')):
            word=link.get('href')
            wordlist=word.split(split, 1)
            if len(wordlist)>1:
              url=split+wordlist[1]
            else:
              url=wordlist  
            self.articles.append(url)
  def getdatelinks1(self, d, m, year):
      months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] 
  
      from selenium.common.exceptions import NoSuchElementException
      page =1 
      month=months[m-1]
      dated='"'+str(d)+'%20'+month+'%20'+str(year)+'"\n'
      f=1 #flage
      j=0
      print(" => getting article links...........", d,month+",",year)
      wd=webdriver.Chrome(executable_path=r"chromedriver", options=options) #Path to Chrome Driver
      url="https://www.scmp.com/search/"
      valid=1
      try:
          wd.get(url)
      except TimeoutException as ex:
          try:
              wd.get(url)
          except TimeoutException as ex:
              wd.get(url)
      #wd.maximize_window()
      time.sleep(1)
      print(url)
      wd.find_element_by_xpath('//*[@id="main-content"]/div[3]/div/div[1]/div/input').send_keys(dated)
      from selenium.webdriver.common.action_chains import ActionChains

      el=wd.find_element_by_xpath('//*[@id="main-content"]/div[3]/div/div[1]/div/input')
      action = ActionChains(wd)
      action.move_to_element(el).click().perform()
      wd.find_element_by_tag_name('body').send_keys(Keys.TAB)
      time.sleep(1)
      wd.find_element_by_tag_name('body').send_keys(Keys.END)
  

      time.sleep(3)
      while(f):        
              f=0
              try:
                  link = wd.find_element_by_class_name('no-results')
              except NoSuchElementException:
                  f=1    
              if f!=0:   
                  alinks1=wd.find_elements_by_tag_name('a')
                  wd.find_element_by_tag_name('body').send_keys(Keys.TAB)
                  wd.find_element_by_tag_name('body').send_keys(Keys.END)
                  time.sleep(3)
                  alinks2=wd.find_elements_by_tag_name('a')
                  if len(alinks1)==len(alinks2):
                      f=0

      alinks =wd.find_elements_by_class_name("item__wrapper")
      for e in alinks:
          f=1
          try:
              date=e.find_element_by_class_name('wrapper__published-date').text
          except NoSuchElementException:
              f=0    
          if f!=0:
              dy=str(d)
              temp=dy+" "+month+" "+str(year)
                  #if (date.find(temp) != -1):
              lnk=e.find_element_by_tag_name('a')
              self.articles.append(lnk.get_attribute('href'))
      wd.close()
  def getarticle(self, x):
      import time
      months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] 
      print(" => downloading..... "+x)
      d=webdriver.Chrome(executable_path=r"chromedriver", options=options) #Path to Chrome Driver
      #d=webdriver.Chrome('chromedriver',options=options) #Path to Chrome Driver
      d.maximize_window()
      valid=1
      try:
          d.get(x)
          d.get(x)
      except TimeoutException as ex:
          print("Exception has been thrown. " + str(ex))
          valid=0
      
      try:
          el=d.find_element_by_class_name('sections__tag')
          el=d.find_element_by_class_name('head__info')
      except NoSuchElementException:
          valid=0
      if valid==1:
          flg=1
          try:
              el=d.find_element_by_class_name('details__read-more')
          except NoSuchElementException:
                  flg=0
          if flg==1:
              try:
                  actions = ActionChains(d)
                  actions.move_to_element(el).perform()
                  actions.perform()
                  actions.move_by_offset(1, 1)
                  actions.click()
                  actions.perform()  
              except exceptions.StaleElementReferenceException:
                  pass
          else:
              try:
                  el=d.find_element_by_class_name('head__info')
                  action = ActionChains(d)
                  action.move_to_element(el).click().perform()
                  d.find_element_by_tag_name('body').send_keys(Keys.TAB)
                  d.find_element_by_tag_name('body').send_keys(Keys.END) 
              except exceptions.StaleElementReferenceException:
                  pass
          source="scmp"
          author=""
          author = d.find_element_by_class_name('article-author__name').text

          link=str(x)
          title = d.find_element_by_class_name('info__headline').text


          text=""
          txt = d.find_elements_by_class_name('content--p') # article-details-type--p content--p
          for t in txt:
              tt=""
              try:
                  tt=t.text
              except exceptions.StaleElementReferenceException:
                  pass
              text=text+"\n"+(tt) 

          #print(text)    
          y = d.find_element_by_class_name('article-author__published-date').text
          a=y.split(', ', 1)
          c=a[1]
          a=c.split(', ', 1)
          b=a[0]
          c=b.split(' ', 1)
          timing=a[1]    
          dy=c[0]
          mt=c[1]
          j=1
          for i in range(0, 13):
              if(mt==months[i]):
                  break
              j=j+1

          year=a[1]
          #print(year, " ", mt, " ", dy)
          DateOfPublication=year+"-"+str(j)+"-"+dy
          DateOfExtraction=str(datetime.date(datetime.now()))
          article=text
          d.close()
          return source, link, DateOfExtraction, DateOfPublication, title, author, article
      else:
          return "0","0","0","0","0","0" ,"0"


  def extract(self, date1, date2, c):
    from datetime import datetime
    Datetoday=str(datetime.date(datetime.now()))
    today=Datetoday.split('-',2)
    year=int(today[0])
    month=int(today[1])
    day=int(today[2])
    day=day-1
    if day==0:
        if(month==3):
            day=28
        if month in [4,5,7,8,10,12]:
            day=30
            
        if month in [1,2,4,6,9,11]: 
            day=31
        month=month-1
        if(month==0):
            month=12


    print("\n => scraper: scmp news", " - date:", Datetoday)
    
    #input1 from: 
    m2=month #month
    d2=day #day
    m1=month #month
    d1=day #day

    #input2 to:
    
  
    if c==0: #defined range
      print(" to:",date1, "----- from:",date2,"\n")
      datei=date1.split('-',2)
      year=int(datei[0])
      m1=int(datei[1])
      d1=int(datei[2])
      datef=date2.split('-',2)
      year=int(datef[0])
      m2=int(datef[1])
      d2=int(datef[2])

    if c==1:  #daily
      m1=m2 #month
      d1=d2 #day
    
    url=""
    chk=0
    for j in range(m1, m2+1):
        df=31
        di=1
        if j==m1:
            di=d1
        if j==m2:
            df=d2
        for i in range(di, df+1):
            self.articles.clear()
            self.downloaddata(i, j, year, url, chk)


### Scrapper: Xinhua News

In [30]:
class Xinhua(NewsExtractor):

  def getdatelinks(self, d, m, year):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'] 
  
    from selenium.common.exceptions import NoSuchElementException
    page =1 
    month=months[m-1]
    dated=month+"%20"+str(d)
    f=1 #flage
    j=0
    print(" => getting article links...........", dated)
    while(f):
        wd=webdriver.Chrome('chromedriver',options=options) #Path to Chrome Driver
        p=str(page)
        url="http://search.news.cn/?lang=en#search/0/"+dated+"/"+ p +"/"
        wd.get(url)
        print(url)
        time.sleep(1)
        f=0
        try:
          link = wd.find_element_by_class_name('noResult')
        except NoSuchElementException:
          f=1  
        if f!=0:  
          #el = wd.find_element_by_xpath('//*[@id="newsCon"]/div[2]/div[1]/div/p[1]/span').text
          #print(el)
          alinks=wd.find_elements_by_class_name('news')
          for lnk in alinks:
            lnks = lnk.find_element_by_tag_name('h2')  
            l = lnks.find_element_by_tag_name('a')
            t=lnk.find_element_by_class_name('easynews')
            tt=t.find_element_by_class_name('newstime')
            date = tt.find_element_by_tag_name('span').text
            dy=str(d)
            mt=str(m)
            if d<10:
              dy="0"+str(d)
            if m<10:
              mt="0"+str(m)
            temp=str(year)+"-"+mt+"-"+dy
            if (date.find(temp) != -1):
              self.articles.append(l.get_attribute('href'))
        page=page+1
        wd.close()
    
            #print(l.get_attribute('href'))
  def getarticle(self, x):
    import time
    print(" => downloading..... "+x)
    d=webdriver.Chrome('chromedriver',options=options) #Path to Chrome Driver
    #d.minimize_window()
    err=0
    try:
      d.get(x)
    except selenium.common.exceptions.WebDriverException:
      err=1
    if err!=1:
      d.get(x)
      
      time.sleep(2)
      source="Xinhua"
      author="huaxia"
      link=str(x)
      title = d.find_element_by_class_name('Btitle').text
  
      text=""
      txt = d.find_element_by_class_name('content')
      all_links = txt.find_elements_by_tag_name('p')
      for i in all_links:
          text=text+"\n"+i.text
      
      y = d.find_element_by_class_name('time').text
      a=y.split(' ', 1)
      a=a[1]
      a=y.split(' ', 1)
      b=a[0]
      timing=a[1]       
      DateOfPublication=b
      DateOfExtraction=str(datetime.date(datetime.now()))
      article=text
      d.close()
      return source, link, DateOfExtraction, DateOfPublication, title, author, article
    else:
      return "0","0","0","0","0","0" ,"0"      

  def extract(self, date1, date2, c):
    from datetime import datetime
    Datetoday=str(datetime.date(datetime.now()))
    today=Datetoday.split('-',2)
    year=int(today[0])
    month=int(today[1])
    day=int(today[2])
    day=day-1
    if day==0:
        if(month==3):
            day=28
        if month in [4,5,7,8,10,12]:
            day=30
            
        if month in [1,2,4,6,9,11]: 
            day=31
        month=month-1
        if(month==0):
            month=12


    print("\n => scraper: xinhua news", " - date:", Datetoday)
    #input1 from: 
    m2=month #month
    d2=day #day
    m1=month
    d1=day

    #input2 to:

    if c==0: #defined range
      print(" to:",date1, "----- from:",date2,"\n")
      datei=date1.split('-',2)
      year=int(datei[0])
      m1=int(datei[1])
      d1=int(datei[2])
      datef=date2.split('-',2)
      year=int(datef[0])
      m2=int(datef[1])
      d2=int(datef[2])

    if c==1:  #daily
      m1=m2 #month
      d1=d2 #day
    
    url=""
    chk=0
    for j in range(m1, m2+1):
        df=31
        di=1
        if j==m1:
            di=d1
        if j==m2:
            df=d2
        for i in range(di, df+1):
            self.articles.clear()
            self.downloaddata(i, j, year, url, chk)

### Scrapper: Tass News

In [31]:
class Tass(NewsExtractor):

  def getdatelinks(self, d, m, y):
      months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] 
      if d<10:
        day="0"+str(d)
      else:
        day=str(d)
      year=str(y)  
      month=months[m-1]
      print(" => getting article links...........", day,month+",",year)
      import requests
      from bs4 import BeautifulSoup
      catagories=["politics", "world", "sport", "economy", "defense" ]

      #check="/article/"
      #check1="/opinion/"
      split="https://tass.com/"
      for c in catagories:
        url="https://web.archive.org/web/"+year+month+str(day)+"032150"+"/https://tass.com/"+c
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for link in soup.find_all("a", {"class": "news-preview"}):
          print(link.get('class'))
          if  str(link.get('href')) not in self.articles:
            word=str(link.get('href'))
            print(word)
            
            wordlist=word.split(split, 1)
            if len(word)==len(wordlist):
              w=wordlist[0]
              print(w)
            else:  
              if len(wordlist)>1:
                print(wordlist)
                url=split+wordlist[1]
                print(url)
              else:
                url=wordlist  
            self.articles.append(url)
            #print(url)
  def getdatelinks1(self, url):
    wd = webdriver.Chrome('chromedriver',options=options)
    wd.get(url)
    wd.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    flag=1
    while flag: 
      l1 = wd.find_elements_by_tag_name('a')
      wd.execute_script("window.scrollTo(0,document.body.scrollHeight)")
      time.sleep(2)
      wd.find_element_by_tag_name('body').send_keys(Keys.END) 
      time.sleep(2)
      l2 = wd.find_elements_by_tag_name('a')
      if(len(l1)==len(l2)):
        flag=0
    #j=0
    all_links = wd.find_elements_by_tag_name('a')
    for i in all_links:
      if (i.get_attribute("class")=="news-preview news-preview_default 987789 ng-scope"):
        #j=j+1
        self.articles.append(i.get_attribute("href"))
        #print(i.get_attribute("href"))
    wd.close()
    
            
  def getarticle(self, x):
    print(" => downloading..... "+x)

    d=webdriver.Chrome('chromedriver', options=options) #Path to Chrome Driver
    err=0
    try:
      d.get(x)
    except selenium.common.exceptions.WebDriverException:
      err=1
    if err!=1:
      source="tass"
      author="tass"
      link=str(x)
      try:
        title = d.find_element_by_class_name('news-header__title').text
      except selenium.common.exceptions.NoSuchElementException:  
        err=1
      if err!=1:
        text=""
        all_links = d.find_elements_by_class_name('text-block')
        for i in all_links:
            text=text+i.text

        months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

        y = d.find_element_by_class_name('news-header__date-date').text
        
        a=y.split(', ', 1)
        b=a[0]
        c=b.split(' ', 1)
        day=c[0]
        str_month=c[1]
        m=0
        for n in months:
            m=m+1
            if str_month==n:
                break
        year="2021"    
        timing=a[1]
        month='0'+str(m)
        DateOfPublication=year+"-"+month+"-"+day
        
        DateOfExtraction=str(datetime.date(datetime.now()))
        
        article=text
        d.close() 
        return source, link, DateOfExtraction, DateOfPublication, title, author, article
      else:
        return "0","0","0","0","0","0" ,"0"   
    else:
      return "0","0","0","0","0","0" ,"0"      

  def extract(self, date1, date2, c):
    from datetime import datetime
    Datetoday=str(datetime.date(datetime.now()))
    print("\n => scraper: tass news", " - date:", Datetoday)



    if c==0: #defined range
      print(" to:",date1, "----- from:",date2,"\n")
      datei=date1.split('-',2)
      year=int(datei[0])
      m1=int(datei[1])
      d1=int(datei[2])
      datef=date2.split('-',2)
      year=int(datef[0])
      m2=int(datef[1])
      d2=int(datef[2])

      url=""
      chk=0
      for j in range(m1, m2+1):
          df=31
          di=1
          if j==m1:
              di=d1
          if j==m2:
              df=d2
          for i in range(di, df+1):
              self.articles.clear()
              self.downloaddata(i, j, year, url, chk)

    else:
      if c==1:  #daily
        url="https://tass.com/search?sort=date&range=day"
      if c==2: 
        url="https://tass.com/search?sort=date&range=week"
      if c==3:  #monthly
        url="https://tass.com/search?sort=date&range=month"
      
      i=0
      j=0
      year=0
      chk=1
      self.articles.clear()  
      self.downloaddata(i, j, year, url, chk)

### Scrapper Object Initilization & Directory Path Set

In [32]:

o1=Tass()
o1.setdirpath("/content/drive/MyDrive/datasets/tass/")
o2=Sputnik()
o2.setdirpath("/content/drive/MyDrive/datasets/sputnik/")
o3=Xinhua()
o3.setdirpath("/content/drive/MyDrive/datasets/xinhua/")
o4=Scmp()
o4.setdirpath("/content/drive/MyDrive/datasets/scmp/")





### Scrapper Exectuation Daily

In [33]:
o1.daily()
o2.daily()
o3.daily()
o4.daily()

 => Daily news article parser

 => scraper: tass news  - date: 2021-07-26
 => links extracted :64
 threade:  =>  0
 threade: 0   => ( 0 / 13 )
 => downloading..... https://tass.com/sport/1317729
 threade:  =>  1
 threade: 1   => ( 0 / 13 )
 => downloading..... https://tass.com/economy/1317683
 threade:  =>  2
 threade: 2   => ( 0 / 13 )
 => downloading..... https://tass.com/politics/1317571
 threade:  =>  3
 threade: 3   => ( 0 / 13 )
 => downloading..... https://tass.com/sport/1317541
 threade:  =>  4
 threade: 4   => ( 0 / 12 )
 => downloading..... https://tass.com/defense/1317503
 threade: 0   => ( 1 / 13 )
 => downloading..... https://tass.com/sport/1317745
 threade: 1   => ( 1 / 13 )
 => downloading..... https://tass.com/defense/1317681
 threade: 2   => ( 1 / 13 )
 => downloading..... https://tass.com/world/1317563
 threade: 3   => ( 1 / 13 )
 => downloading..... https://tass.com/sport/1317539
 threade: 4   => ( 1 / 12 )
 => downloading..... https://tass.com/economy/1317499
 threa

### Scrapper Exectuation In Date Range 

In [34]:

#date1: FROM date to start news etraction
#date2: TO date to end news etraction
date1="2021-07-26"    
date2="2021-07-26"

#Call parser objects in date-range
o1.rangedate(date1, date2)
o2.rangedate(date1, date2)
o3.rangedate(date1, date2)
o4.rangedate(date1, date2)

 => specified news article parser

 => scraper: tass news  - date: 2021-07-26
 to: 2021-07-26 ----- from: 2021-07-26 

 => getting article links........... 26 07, 2021
['news-preview', 'news-preview_default']
/web/20210722140252/https://tass.com/politics/1316703
['/web/20210722140252/', 'politics/1316703']
https://tass.com/politics/1316703
['news-preview', 'news-preview_default']
/web/20210722140252/https://tass.com/politics/1316693
['/web/20210722140252/', 'politics/1316693']
https://tass.com/politics/1316693
['news-preview', 'news-preview_default']
/web/20210722140252/https://tass.com/politics/1316687
['/web/20210722140252/', 'politics/1316687']
https://tass.com/politics/1316687
['news-preview', 'news-preview_default']
/web/20210722140252/https://tass.com/politics/1316677
['/web/20210722140252/', 'politics/1316677']
https://tass.com/politics/1316677
['news-preview', 'news-preview_default']
/web/20210722140329/https://tass.com/politics/1316703
['/web/20210722140329/', 'politics/131670

Exception in thread Thread-34:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.7/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.7/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.7/http/client.py", line 288, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (m

KeyboardInterrupt: ignored

### Scrapper Exectuation Weekly

In [38]:
o1.weekly()
o2.weekly()
o3.weekly()
o4.weekly()

 => weekly news article parser
 ---- 2021-07-26 -----

 => specified news article parser

 => scraper: scmp news  - date: 2021-07-26
 to: 2021-7-18 ----- from: 2021-7-25 

 => getting article links........... 18 07, 2021


KeyboardInterrupt: ignored

### Scrapper Exectuation Monthly

In [39]:
o1.monthly()
o2.monthly()
o3.monthly()
o4.monthly()

 => monthly news article parser
 => specified news article parser

 => scraper: tass news  - date: 2021-07-26
 to: 2021-6-25 ----- from: 2021-7-25 

 => getting article links........... 25 06, 2021
['news-preview', 'news-preview_default']
/web/20210624152400/https://tass.com/politics/1306945
['/web/20210624152400/', 'politics/1306945']
https://tass.com/politics/1306945
['news-preview', 'news-preview_default']
/web/20210624152400/https://tass.com/politics/1306941
['/web/20210624152400/', 'politics/1306941']
https://tass.com/politics/1306941
['news-preview', 'news-preview_default']
/web/20210624152400/https://tass.com/politics/1306939
['/web/20210624152400/', 'politics/1306939']
https://tass.com/politics/1306939
['news-preview', 'news-preview_default']
/web/20210624152400/https://tass.com/politics/1306933
['/web/20210624152400/', 'politics/1306933']
https://tass.com/politics/1306933
['news-preview', 'news-preview_default']
/web/20210624152447/https://tass.com/world/1306925
['/web/2021062

KeyboardInterrupt: ignored